In [2]:
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import load_model
import tensorflow_hub as hub

In [4]:
def Predict_CXR(image_path, feature_model_path, model_path):
    # 모델 불러오기
    # 모델 불러올 때 Warning이 표시되는데 이는 더 training하지 못한다는 의미이고,
    # 이 함수에서는 더 이상 training을 하지 않음.
    model = load_model(model_path)
    feature_model = load_model(feature_model_path, custom_objects={'KerasLayer': hub.KerasLayer})

    # 이미지를 불러옴
    img_path= image_path
    img = cv2.imread(img_path)
    # 이미지 크기를 모델에 맞게 조정
    img = cv2.resize(img, dsize=(224,224))
    # 이미지 픽셀값 스케일링
    img = img / 255.0
    # 이미지 차원 확장 (224,224,3) -> (1,224,224,3)  : feature_model에 넣기위함
    img = np.expand_dims(img, axis=0)
    # feature_model에서 feature 추출
    feature_vector = feature_model.predict(img)
    # 앞서 생성한 model 분류기를 통해 예측 수행
    prediction = model.predict(feature_vector)[0]

    unique_sorted_Y = ['COVID19','NORMAL','PNEUMONIA']
    #확률의 예측값을 5개 선출 
    #[::-1] Extended Slices 기법으로 내림차순으로 변경
    top_3_predict = prediction.argsort()[::-1]

    # labels에 저장 
    labels = [unique_sorted_Y[index] for index in top_3_predict]
    print(labels[0])
    # #-----------------------------------------#
    # # CXR 이미지와 예측확률 plot
    # plt.figure(figsize=(20,5))
    # plt.subplot(1,2,1)
    # plt.imshow(plt.imread(img_path)) # 원본 이미지를 가져온다.

    # plt.subplot(1,2,2)
    # color = ['blue'] * 3

    # #가로 막대 그리기 (확률값)
    # plt.barh(range(3), prediction[top_3_predict][::-1] * 100, color = color)
    # # 상태 y축
    # plt.yticks(range(3), labels[::-1])

    # plt.show()

Predict_CXR('/content/drive/My Drive/인공지능 학습관련/COVID-19 Radiography Database/data/COVID-19/COVID-19 (1).png',
            '/content/drive/My Drive/인공지능 학습관련/feature_model.h5',
            '/content/drive/My Drive/인공지능 학습관련/my_model.h5')

COVID19


In [5]:
paths = ['/content/F63AB6CE-1968-4154-A70F-913AF154F53D.jpeg','/content/FE9F9A5D-2830-46F9-851B-1FF4534959BE.jpeg',
         '/content/IM-0129-0001.jpeg','/content/IM-0131-0001.jpeg',
         '/content/person905_bacteria_2830.jpeg','/content/person905_virus_1561.jpeg']


for path in paths:
    Predict_CXR(path,
            '/content/drive/My Drive/인공지능 학습관련/feature_model.h5',
            '/content/drive/My Drive/인공지능 학습관련/my_model.h5')

error: ignored